In [ ]:
# !pip3 install matplotlib
# !pip3 install tabula-py
# !pip3 install PyPDF2
# !pip3 install pdfminer.six
# !pip3 install PyMuPDF
# !pip3 install camelot-py
# !pip3 install --upgrade pip
# !pip3 install geopandas
# !pip3 install plotly
# !pip3 install pandas
# !pip3 install pdfplumber
# !pip3 install scipy
# !pip3 install --upgrade pandas
# %pip install arcgis
# !pip3 install json
# !pip3 install gtfs_functions
# !pip3 install keplergl
# %pip install Ipython

In [2]:
import pandas as pd
import numpy as np
import matplotlib as mpt
import os
import geopandas as gpd
import fiona
from glob import glob
import plotly.express as px
from shapely.geometry import Point, MultiPolygon
from arcgis.gis import GIS
from arcgis.features.analysis import summarize_within
from IPython.display import display
from gtfs_functions import Feed
from gtfs_functions.gtfs_plots import map_gdf
# import keplergl as kpl


/Users/sulemanbasit/Library/Python/3.9/lib/python/site-packages/dask/dataframe/__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
/Users/sulemanbasit/Library/Python/3.9/lib/python/site-packages/dask/dataframe/__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:

def save_to_csv(df, output_folder, file_name):
    """AI is creating summary for save_to_csv

    Args:
        df (Pandas Data Frame): Contains data frame that needs to be converted and saved as csv
        output_folder (Str): Destination path where df needs to be saved
        file_name (Str): File name that df needs to be saved as
    """
    # Check if the output folder exists, if not, create it
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Define the output file path
    output_file_path = os.path.join(output_folder, file_name)
    
    # Save the DataFrame to CSV in the output folder
    df.to_csv(output_file_path, index=False)
    print(f"DataFrame saved to: {output_file_path}")


In [ ]:
def convert_data (file_path):
    """
    Function to convert files of appropriate structure to csv format, creating point object with files that have lat and lon values

    Args:
        file_path (str): File Path to open and convert to csv file format
    """
    df = pd.read_csv(file_path)
    output_folder_path = os.getcwd()+"/Data Sets/Analysis Data/"
    # print(output_folder_path)
    output_file_name = file_path.split('/')[-1].split('.')[0] + ".csv"
    save_to_csv(df, output_folder_path, output_file_name)
    print("{} saved in {}".format(output_file_name, output_folder_path))

In [ ]:
data_path = os.getcwd()+"/Data Sets/CT_GTFS/"
# ignore_file = ['agency.txt', "calendar.txt","calendar_dates.txt"]
convert_data(data_path+"shapes.txt")

# Program To Calculate Demand Index Using Z-Method

In [ ]:
community_data_path = os.getcwd()+"/Data Sets/Analysis Data/Community Profiles Compiled.xlsx"

In [ ]:
comm_df = pd.read_excel(community_data_path)
# comm_df.head()

In [ ]:
comm_df.head()

In [ ]:
output_folder_path = os.getcwd()+"/Data Sets/Analysis Data"

In [ ]:
#Z-Score method manually
pop_mean = comm_df['Population in private households'].mean()
pop_std = comm_df['Population in private households'].std()


medinc_mean = comm_df["Median household income of private households"].mean()
medinc_std = comm_df["Median household income of private households"].std()

low_mean = comm_df['Population in private households to whom low income concepts are applicable (Number in low income)'].mean()
low_std = comm_df['Population in private households to whom low income concepts are applicable (Number in low income)'].std()

trans_mean = ((comm_df['Employed']/comm_df["Employed labour force aged 15 years and over in private households"])*comm_df['Public transit']).mean()
trans_std = ((comm_df['Employed']/comm_df["Employed labour force aged 15 years and over in private households"])*comm_df['Public transit']).std()

rent_mean = (comm_df['Per cent households with income spending 30% or more total income on shelter (Renter)']*comm_df['Private households with total income greater than zero (Renter)']).mean()
rent_std = (comm_df['Per cent households with income spending 30% or more total income on shelter (Renter)']*comm_df['Private households with total income greater than zero (Renter)']).std()

seniors_mean = comm_df['65 to 84 years'].mean()
seniors_std = comm_df['65 to 84 years'].std()

In [ ]:
row, col = comm_df.shape
z_rows_to_append = []
for c in range(row):
    community = comm_df['Community Name'][c]
    
    pop = comm_df['Population in private households'][c]
    # pop_ss = (pop-pop_max)/(pop_max-pop_min)
    z_pop_score = (pop-pop_mean)/pop_std
    
    med = comm_df['Median household income of private households'][c]
    # med_ss = (med-medinc_max)/(medinc_max-medinc_min)
    z_med_score = (med-medinc_mean)/medinc_std
    
    trans = ((comm_df['Employed']/comm_df["Employed labour force aged 15 years and over in private households"])*comm_df['Public transit'])[c]
    # trans_ss = (trans-trans_max)/(trans_max-trans_min)
    z_trans_score = (trans-trans_mean)/trans_std
    
    rent = (comm_df['Per cent households with income spending 30% or more total income on shelter (Renter)']*comm_df['Private households with total income greater than zero (Renter)'])[c]
    # rent_ss = (rent-rent_max)/(rent_max-rent_min)
    z_rent_score = (rent-rent_mean)/rent_std
    
    low = comm_df['Population in private households to whom low income concepts are applicable (Number in low income)'][c]
    # low_ss = (low-low_max)/(low_max-low_min)
    z_low_score = (low-low_mean)/low_std

    sen = comm_df['65 to 84 years'][c]
    z_seniors_score = (sen-seniors_mean)/seniors_std

    z_result = z_trans_score+z_rent_score+z_low_score-z_med_score+z_seniors_score
    # z_result = z_trans_score+z_rent_score+z_low_score+z_seniors_score

    column_df = {"Community Name": community, "Z Score": z_result}
    columns_df = {"Community Name": community, "Low Income Index": z_low_score, "Seniors Index": z_seniors_score, "Public Transit Index": z_trans_score, "Rent Index": z_rent_score, "Median Income": z_med_score}
    z_rows_to_append.append(column_df)


Z_index_df = pd.DataFrame(z_rows_to_append)

In [ ]:
Z_index_df.head()

In [ ]:
z_max = Z_index_df['Z Score'].max()
z_min = Z_index_df['Z Score'].min()

In [ ]:
print(z_max)
print(z_min)

In [ ]:
row, col = Z_index_df.shape
z_ss_rows = []
for c in range(row):
    community = Z_index_df["Community Name"][c]
    z_score = Z_index_df['Z Score'][c]

    z_ss_result = (z_score-z_min)/(z_max-z_min)

    column_df = {"Community Name": community, "Z Score": z_ss_result}
    z_ss_rows.append(column_df)
    
z_ss_index_df = pd.DataFrame(z_ss_rows)

In [ ]:
z_ss_index_df.head()

In [ ]:
save_to_csv(z_ss_index_df, output_folder_path, 'Z Standardized Demand Index.csv')

# Program To Visualize ArcGis Data

In [ ]:
gis_link = "https://ucalgary.maps.arcgis.com"
gis_user = "sbasit_ucalgary"
gis_user2 = "suleman.basit@ucalgary.ca_ucalgary"
gis_pass2 = "Dra90nK!n9"
gis_pass = "Sa5nar9("
gis_key = "AAPK049cf86d96f4462ba33f8b661792d2f0z748Z0jNex7rIOx45MLoRtWEUFk5CElu0-Obz_3bxUwj9_Q8MI398NcMkS9xWe5F"
stop_freq_id = "ec5537fa5cca45a7ba53f3d70a961c9e"
comm_boundary_id = "23864b8797424379951a0d4117034bdb"

In [ ]:
layers=[]

In [ ]:
my_gis = GIS(gis_link,gis_user,gis_pass,gis_key)

In [ ]:
my_content = my_gis.content.search(query= "*",item_type='Feature *')
my_content

In [ ]:
# stop_map = my_gis.content.get(stop_freq_id)
my_group = my_gis.groups.search(query='owner:'+gis_user2)[0]

# stop_map.add_layer(comm_map)
# map.add_layer(comm_boundary_id)

In [ ]:
for g in my_group.content():
    print(g)

In [ ]:
group_content = my_gis.groups.get(my_group.groupid)

In [ ]:
# group_content.search(query='owner:'+gis_user2+" and description: Community")['results']
for i in group_content.search(query='title: Stops Sorted')['results']:
    # if "Stops" in i and "Layer" in i:
    layers.append(i)

In [ ]:
layers

In [ ]:
map = my_gis.map('Calgary,AB')
stop = my_gis.content.get(stop_freq_id)
map.add_layer(stop)
# comm_map
# print("echo")

# Join Multipolygon Data To Profile

In [ ]:
folder_path = os.getcwd()+"/Data Sets/Analysis Data/"
comm_file = "Community Profiles Compiled.xlsx"
bound_file = "Boundraries_Data_by_Community .csv"
z_index_file = "Z Standardized Demand Index.csv"
census_file = "Census by Community 2019_20240401.geojson"

In [ ]:
comm_data = pd.read_excel(folder_path+comm_file)
comm_bound = gpd.read_file(folder_path+census_file)
z_index = pd.read_csv(folder_path+z_index_file)

In [ ]:
comm_bound.columns

In [ ]:
comm_bound_lower = comm_bound.copy()
comm_bound_lower['name'] = comm_bound_lower['name'].str.lower()
comm_data_lower = comm_data.copy()
comm_data_lower['Community Name'] = comm_data_lower['Community Name'].str.lower()
comm_z_lower = z_index.copy()
comm_z_lower["Community Name"] = comm_z_lower['Community Name'].str.lower()

# Merge the DataFrames based on the lower case names
# merged_data = pd.merge(comm_data_lower, comm_bound_lower, left_on='Community Name', right_on='NAME', how='inner')
merged_data = pd.merge(comm_z_lower, comm_bound_lower, how='left', left_on='Community Name', right_on='name')

# Convert back to original case if needed
merged_data['name'] = comm_bound['name']  # Assuming you want to keep the original case for boundary names
merged_data['Community Name'] = comm_data['Community Name']  # Assuming you want to keep the original case for community names

# Now merged_data contains the rows where the names match, regardless of case

In [ ]:
merged_data.head()

In [ ]:
sub_set_data = merged_data[["Community Name","Z Score", "geometry"]]

In [ ]:
sub_set_data.head()
type(sub_set_data['geometry'][1])

In [ ]:
gpd_data = gpd.GeoDataFrame(sub_set_data, geometry="geometry")

In [ ]:
gpd_data.to_file(folder_path+"Community_Z_Boundary.geojson", driver="GeoJSON")

# Generate Data Sets For Calculating Supply Frequency

In [6]:
gtfs_path = os.getcwd()+"/Data Sets/CT_GTFS.zip"
# feed = Feed(gtfs_path, start_date = "2024-02-19", end_date = "2024-02-23", time_windows=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24])
feed = Feed(gtfs_path, start_date = "2024-02-19", end_date = "2024-02-23", time_windows=[0,5,6,9,15,18,24])

In [7]:
routes = feed.routes
trips = feed.trips
stops = feed.stops
stop_times = feed.stop_times
shapes = feed.shapes

INFO:root:Reading "routes.txt".
INFO:root:accessing trips
INFO:root:Reading "trips.txt".
INFO:root:Reading "calendar.txt".
INFO:root:Reading "calendar_dates.txt".
INFO:root:The busiest date/s of this feed or your selected date range is/are:  ['2024-02-19'] with 17641 trips.
INFO:root:In the case that more than one busiest date was found, the first one will be considered.
INFO:root:In this case is 2024-02-19.
INFO:root:Reading "stop_times.txt".
INFO:root:_trips is defined in stop_times
INFO:root:Reading "stops.txt".
INFO:root:computing patterns
INFO:root:Reading "shapes.txt".


In [8]:
# feed = Feed(gtfs_path,start_date="2024-02-17", end_date="2024-02-17")
stop_freq = feed.stops_freq

In [9]:
stop_list = [3641, 3816, 3817, 3960, 4155, 4934, 6801, 6802, 6803, 6804, 6805, 6806, 6807, 6815, 6816, 6817, 6818, 6819, 6820, 6822, 6823, 6824, 6825, 6827, 6828, 6829, 6830, 6831, 8556, 8557, 8558, 8559, 8560, 8561, 8562, 8563, 8564, 8565, 8566, 9261, 9262, 9263, 9264, 9387, 9390, 9391, 9392, 3627, 3628, 3629, 3630, 3631, 3632, 3633, 3634, 3635, 3636, 3637, 3638, 3639, 6747, 6808, 6809, 6810, 6811, 6812, 6813, 6814, 6825, 6831, 8567, 8568, 8569, 8570, 8571, 8572, 9385, 9396, 9781, 9896, 9897, 2350, 2376, 2793, 2794, 2795, 2796, 2797, 2798, 2799, 2800, 2801, 2802, 2803, 3495, 5701, 5703, 5718, 5730, 5731, 5732, 5734, 5789, 6048, 6724, 6846, 7281, 7759, 7762, 7763, 7764, 7881, 7892, 8157, 8161, 8162, 8163, 8215, 9400, 9407, 9409, 9430, 9439, 9617, 3368, 3370, 3371, 3372, 3380, 3381, 3382, 3383, 3384, 3751, 3761, 4567, 4568, 4582, 4583, 5147, 5385, 5630, 5752, 5754, 5767, 5769, 6099, 6110, 6111, 6586, 6982, 8038, 8368, 9174, 9178, 9828, 2469, 2486, 2487, 2488, 2489, 2498, 3371, 3531, 3744, 3745, 3746, 3747, 3748, 3755, 3756, 3764, 3765, 3766, 3767, 3870, 5762, 5763, 6119, 6982, 9453, 9935, 2178, 2180, 2234, 2299, 2788, 3387, 3388, 3389, 3390, 3391, 3392, 3886, 4352, 5057, 7380, 8271, 8653, 8832, 9802, 9805, 9807, 9808, 9809, 9810, 9815, 9816, 9817, 9818, 9820, 9821]
stop_list_str = [str(x) for x in stop_list]
# stop_freq.loc[stop_freq["stop_id"].isin(stop_list_str)].sort_values(by='stop_id').to_excel("PTN_stop_frequency.xlsx")
stop_freq.loc[stop_freq["stop_id"].isin(stop_list_str)].sort_values(by='stop_id')
# type(stop_freq["stop_id"][1])
# stop_freq

,stop_id,direction_id,window,ntrips,min_per_trip,stop_name,geometry
998,2178,0,0:00-5:00,2,150,EB 17th AV SE @ 70th ST SE,POINT (-113.92965 51.03769)
999,2178,0,15:00-18:00,19,9,EB 17th AV SE @ 70th ST SE,POINT (-113.92965 51.03769)
1000,2178,0,18:00-24:00,27,13,EB 17th AV SE @ 70th ST SE,POINT (-113.92965 51.03769)
1001,2178,0,5:00-6:00,1,60,EB 17th AV SE @ 70th ST SE,POINT (-113.92965 51.03769)
1002,2178,0,6:00-9:00,19,9,EB 17th AV SE @ 70th ST SE,POINT (-113.92965 51.03769)
...,...,...,...,...,...,...,...
37269,9935,0,6:00-9:00,15,12,Douglas Glen Boulevard Station (NB),POINT (-114.00778 50.95750)
37266,9935,0,15:00-18:00,15,12,Douglas Glen Boulevard Station (NB),POINT (-114.00778 50.95750)
37267,9935,0,18:00-24:00,17,21,Douglas Glen Boulevard Station (NB),POINT (-114.00778 50.95750)
37268,9935,0,5:00-6:00,1,60,Douglas Glen Boulevard Station (NB),POINT (-114.00778 50.95750)


In [10]:
line_freq = feed.lines_freq

In [11]:
# line_freq.loc[line_freq["route_name"].str.contains("Line|MAX")].sort_values(by=['window']).to_excel("Lines_Frequency.xlsx")
ptn = line_freq.loc[line_freq["route_name"].str.contains("Line|MAX")].sort_values(by=['window'])

In [12]:
morning_rush = '6:00-9:00'
evening_rush = '15:00-18:00'

In [13]:
rush_hour_performance = ptn.loc[ptn['window'].str.contains(morning_rush+"|"+evening_rush)].sort_values(by=['route_name'])

In [14]:
ptn.shape

(121, 7)

In [15]:
ptn.to_excel('Lines_Frequency.xlsx')

In [16]:
# sum_trips = ptn.groupby('route_name')['ntrips'].sum()
n_trips = rush_hour_performance.groupby('route_id')['ntrips'].sum()

In [17]:
n_trips

route_id
201-20718    226
202-20718    193
303-20727     98
304-20727     75
306-20727     61
307-20727     66
Name: ntrips, dtype: int64

In [18]:
mean_min_per_trip = rush_hour_performance.groupby('route_id')['min_per_trip'].mean()

In [22]:
median_min_per_trip = rush_hour_performance.groupby('route_id')['min_per_trip'].median()

# Visualize through Kepler

In [ ]:
feed = Feed(gtfs_path,start_date="2024-02-17", end_date="2024-02-17")

feed.stop_times.fillna('', inplace=True)
feed.trips.fillna('', inplace=True)
# Repeat for other DataFrame attributes as needed

# Now call get_segments_freq()
segments_freq = feed.segments_freq

In [ ]:
filtered_segments = segments_freq.loc[segments_freq["route_name"].str.contains("MAX|Line")]

In [ ]:
map = kpl.KeplerGl(data=dict(data = filtered_segments, name = "PTN_segments"), config= ptn_config, height = 750)
map.add_data(gpd_data, name="Community")
map

In [ ]:
ptn_config = map.config

# Supply Frequency and Capacity

In [ ]:
data_path = os.getcwd()+"/Data Sets/Analysis Data/"
current_supply_data = "Supplyside588.xlsx"
future_supply_data = "Supplyside588 - With future stops Updated.xlsx"

In [ ]:
blue_trips = 193
blue_freq = 8.9333
red_trips = 444
red_freq = 6.7
orange_trips = 98
orange_freq = 7.0833
teal_trips = 65
teal_freq = 15.6923
yellow_trips = 75
yellow_freq = 9.75
purple_trips = 127
purple_freq = 20.5172

bus_capacity = 65
train_capacity = 600


Current Supply

In [ ]:
supply_df = pd.read_excel(data_path+current_supply_data, sheet_name="Frequency Index")


In [ ]:
supply_df.shape

In [ ]:
sup_fc = pd.DataFrame()

In [ ]:
sup_fc['Community'] = supply_df['Community']
sup_fc['Blue Trips'] = supply_df['Blue']
sup_fc['Blue Freq'] = supply_df['Blue']
sup_fc['Red Trips'] = supply_df["Red"]
sup_fc['Red Freq'] = supply_df["Red"]
sup_fc['Teal Trips'] = supply_df["Teal"]
sup_fc['Teal Freq'] = supply_df["Teal"]
sup_fc['Orange Trips'] = supply_df["Orange"]
sup_fc['Orange Freq'] = supply_df["Orange"]
sup_fc['Yellow Trips'] = supply_df["Yellow"]
sup_fc['Yellow Freq'] = supply_df["Yellow"]
sup_fc['Purple Trips'] = supply_df["Purple"]
sup_fc['Purple Freq'] = supply_df["Purple"]

In [ ]:
sup_fc.loc[(sup_fc['Blue Trips'] != 0),'Blue Trips'] = blue_trips
sup_fc.loc[(sup_fc['Blue Freq'] != 0),'Blue Freq'] = blue_freq
sup_fc.loc[(sup_fc['Red Trips'] != 0),'Red Trips'] = red_trips
sup_fc.loc[(sup_fc['Red Freq'] != 0),'Red Freq'] = red_freq
sup_fc.loc[(sup_fc['Orange Trips'] != 0),'Orange Trips'] = orange_trips
sup_fc.loc[(sup_fc['Orange Freq'] != 0),'Orange Freq'] = orange_freq
sup_fc.loc[(sup_fc['Yellow Trips'] != 0),'Yellow Trips'] = yellow_trips
sup_fc.loc[(sup_fc['Yellow Freq'] != 0),'Yellow Freq'] = yellow_freq
sup_fc.loc[(sup_fc['Teal Trips'] != 0),'Teal Trips'] = teal_trips
sup_fc.loc[(sup_fc['Teal Freq'] != 0),'Teal Freq'] = teal_freq
sup_fc.loc[(sup_fc['Purple Trips'] != 0),'Purple Trips'] = purple_trips
sup_fc.loc[(sup_fc['Purple Freq'] != 0),'Purple Freq'] = purple_freq

In [ ]:
sup_fc.head()

In [ ]:
sup_fc.shape

In [ ]:
comm_data.shape

In [ ]:
# Merge the two DataFrames on the 'Community' and 'Community Name' columns
merged_data = pd.merge(sup_fc, comm_data, left_on='Community', right_on='Community Name', how='right')

# Update the 'Population' column in sup_fc with the corresponding values from comm_data
sup_fc['Population'] = merged_data['Population in private households']

In [ ]:
sup_fc.shape

In [ ]:
freq_col = sup_fc.filter(like="Freq")
sup_fc['Total Freq'] = freq_col.sum(axis=1)

In [ ]:
trip_col = sup_fc.filter(like="Trips")
train_col = trip_col.filter(regex="Blue|Red")
bus_col = trip_col.filter(regex="Orange|Teal|Yellow|Purple")
pop_col = sup_fc.filter(like="Population")

In [ ]:
# sup_fc["Capacity Index"] = ((train_col.sum(axis=1)*train_capacity)+(bus_col.sum(axis=1)*bus_capacity))/pop_col.sum(axis=1)
sup_fc['Sum Train Capacity'] = train_col.sum(axis=1)*train_capacity
sup_fc['Sum Bus Capacity'] = bus_col.sum(axis=1)*bus_capacity

In [ ]:
sup_fc["Capacity Index"] = (sup_fc['Sum Train Capacity']+sup_fc['Sum Bus Capacity'])/pop_col.sum(axis=1)

In [ ]:
min_freq = min(sup_fc["Total Freq"])
max_freq = max(sup_fc["Total Freq"])
min_cap = min(sup_fc['Capacity Index'])
max_cap = max(sup_fc['Capacity Index'])

In [ ]:
print(min_freq, max_freq, min_cap, max_cap)

In [ ]:
sup_fc['SS Freq'] = (sup_fc['Total Freq']-min_freq)/(max_freq-min_freq)
sup_fc['SS Cap'] = (sup_fc['Capacity Index']-min_cap)/(max_cap-min_cap)

In [ ]:
sup_fc.loc[sup_fc["Capacity Index"]==max_cap].head()

In [ ]:
sup_fc.shape

In [ ]:
sup_fc.to_csv(data_path+'Current_Freq_Cap.csv')

Future Supply

In [ ]:
fut_sup_df = pd.read_excel(data_path+future_supply_data, sheet_name = 'F.Frequ')

In [ ]:
sup_fc = pd.DataFrame()

In [ ]:
sup_fc['Community'] = fut_sup_df['Community']
sup_fc['Blue Trips'] = fut_sup_df['Blue']
sup_fc['Blue Freq'] = fut_sup_df['Blue']
sup_fc['Red Trips'] = fut_sup_df["Red"]
sup_fc['Red Freq'] = fut_sup_df["Red"]
sup_fc['Green Trips'] = fut_sup_df["Green"]
sup_fc['Green Freq'] = fut_sup_df["Green"]
sup_fc['Teal Trips'] = fut_sup_df["Teal"]
sup_fc['Teal Freq'] = fut_sup_df["Teal"]
sup_fc['Orange Trips'] = fut_sup_df["Orange"]
sup_fc['Orange Freq'] = fut_sup_df["Orange"]
sup_fc['Yellow Trips'] = fut_sup_df["Yellow"]
sup_fc['Yellow Freq'] = fut_sup_df["Yellow"]
sup_fc['Purple Trips'] = fut_sup_df["Purple"]
sup_fc['Purple Freq'] = fut_sup_df["Purple"]
sup_fc['East Gray Trips'] = fut_sup_df["Teal"]
sup_fc['East Gray Freq'] = fut_sup_df["Teal"]
sup_fc['West Gray Trips'] = fut_sup_df["Teal"]
sup_fc['West Gray Freq'] = fut_sup_df["Teal"]
sup_fc['Future Brentwood Trips'] = fut_sup_df["Teal"]
sup_fc['Future Brentwood Freq'] = fut_sup_df["Teal"]
sup_fc['Future Tuscany Trips'] = fut_sup_df["Teal"]
sup_fc['Future Tuscany Freq'] = fut_sup_df["Teal"]

In [ ]:
print(sup_fc.shape)
sup_fc.head()

In [ ]:
sup_fc.loc[(sup_fc['Blue Trips'] != 0),'Blue Trips'] = blue_trips
sup_fc.loc[(sup_fc['Blue Freq'] != 0),'Blue Freq'] = blue_freq
sup_fc.loc[(sup_fc['Red Trips'] != 0),'Red Trips'] = red_trips
sup_fc.loc[(sup_fc['Red Freq'] != 0),'Red Freq'] = red_freq
sup_fc.loc[(sup_fc['Orange Trips'] != 0),'Orange Trips'] = orange_trips
sup_fc.loc[(sup_fc['Orange Freq'] != 0),'Orange Freq'] = orange_freq
sup_fc.loc[(sup_fc['Yellow Trips'] != 0),'Yellow Trips'] = yellow_trips
sup_fc.loc[(sup_fc['Yellow Freq'] != 0),'Yellow Freq'] = yellow_freq
sup_fc.loc[(sup_fc['Teal Trips'] != 0),'Teal Trips'] = teal_trips
sup_fc.loc[(sup_fc['Teal Freq'] != 0),'Teal Freq'] = teal_freq
sup_fc.loc[(sup_fc['Purple Trips'] != 0),'Purple Trips'] = purple_trips
sup_fc.loc[(sup_fc['Purple Freq'] != 0),'Purple Freq'] = purple_freq
sup_fc.loc[(sup_fc['Green Trips'] != 0), 'Green Trips'] = red_trips
sup_fc.loc[(sup_fc['Green Freq'] != 0), 'Green Freq'] = red_freq
sup_fc.loc[(sup_fc['East Gray Trips'] != 0), 'East Gray Trips']  = teal_trips
sup_fc.loc[(sup_fc['East Gray Freq'] != 0), 'East Gray Freq']  = teal_freq
sup_fc.loc[(sup_fc['West Gray Trips'] != 0), 'West Gray Trips'] = teal_trips
sup_fc.loc[(sup_fc['West Gray Freq'] != 0), 'West Gray Freq']  = teal_freq
sup_fc.loc[(sup_fc['Future Brentwood Trips'] !=0), 'Future Brentwood Trips'] = teal_trips
sup_fc.loc[(sup_fc['Future Brentwood Freq'] !=0), 'Future Brentwood Freq'] = teal_freq
sup_fc.loc[(sup_fc['Future Tuscany Trips'] !=0), 'Future Tuscany Trips'] = teal_trips
sup_fc.loc[(sup_fc['Future Tuscany Freq'] !=0), 'Future Tuscany Freq'] = teal_freq

In [ ]:
print(sup_fc.shape)
sup_fc.head()

In [ ]:
# Merge the two DataFrames on the 'Community' and 'Community Name' columns
merged_data = pd.merge(sup_fc, comm_data, left_on='Community', right_on='Community Name', how='right')

# Update the 'Population' column in sup_fc with the corresponding values from comm_data
sup_fc['Population'] = merged_data['Population in private households']

In [ ]:
sup_fc.shape

In [ ]:
freq_col = sup_fc.filter(like="Freq")
trip_col = sup_fc.filter(like="Trips")
train_col = trip_col.filter(regex="Blue|Red|Green")
bus_col = trip_col.filter(regex="Orange|Teal|Yellow|Purple|Gray|Future")
pop_col = sup_fc.filter(like="Population")

In [ ]:
pop_col.head()

In [ ]:
sup_fc['Total Freq'] = freq_col.sum(axis=1)
sup_fc['Sum Train Capacity'] = train_col.sum(axis=1)*train_capacity
sup_fc['Sum Bus Capacity'] = bus_col.sum(axis=1)*bus_capacity
sup_fc["Capacity Index"] = (sup_fc['Sum Train Capacity']+sup_fc['Sum Bus Capacity'])/pop_col.sum(axis=1)

In [ ]:
min_freq = min(sup_fc["Total Freq"])
max_freq = max(sup_fc["Total Freq"])
min_cap = min(sup_fc['Capacity Index'])
max_cap = max(sup_fc['Capacity Index'])

In [ ]:
sup_fc['SS Freq'] = (sup_fc['Total Freq']-min_freq)/(max_freq-min_freq)
sup_fc['SS Cap'] = (sup_fc['Capacity Index']-min_cap)/(max_cap-min_cap)

In [ ]:
print(sup_fc.shape)
sup_fc.head()

In [ ]:
sup_fc.to_csv(data_path+'Future_Freq_Cap.csv')

# SF SC Through Stop Sorted

In [118]:
data_path = os.getcwd()+"/Data Sets/Analysis Data/"
current_stops_data = "Stops Sorted.xlsx"
future_stops_data = "Stops Sorted Future Network.xlsx"
# comm_file = "Community Profiles Compiled.xlsx"
comm_file = 'demand.csv'

In [119]:
red_code = '201-20718'
blue_code = '202-20718'
orange_code = '303-20727'
yellow_code = '304-20727'
teal_code = '306-20727'
purple_code = '307-20727'

In [120]:
blue_trips = n_trips.loc[blue_code]
blue_freq = median_min_per_trip.loc[blue_code]
red_trips = n_trips.loc[red_code]
red_freq = median_min_per_trip.loc[red_code]
orange_trips = n_trips.loc[orange_code]
orange_freq = median_min_per_trip.loc[orange_code]
teal_trips = n_trips.loc[teal_code]
teal_freq = median_min_per_trip.loc[teal_code]
yellow_trips = n_trips.loc[yellow_code]
yellow_freq = median_min_per_trip.loc[yellow_code]
purple_trips = n_trips.loc[purple_code]
purple_freq = median_min_per_trip.loc[purple_code]

bus_capacity = 65
train_capacity = 600


In [121]:
green_trips = n_trips.loc[red_code]
green_freq = median_min_per_trip.loc[red_code]
east_grey_trips = n_trips.loc[teal_code]
east_grey_freq = median_min_per_trip.loc[teal_code]
west_grey_trips = n_trips.loc[teal_code]
west_grey_freq = median_min_per_trip.loc[teal_code]
future_brentwood_trips = n_trips.loc[teal_code]
future_brentwood_freq = median_min_per_trip.loc[teal_code]
future_tuscany_trips = n_trips.loc[teal_code]
future_tuscany_freq = median_min_per_trip.loc[teal_code]

In [122]:
comm_data = pd.read_csv(data_path+comm_file)
stop_df = pd.read_excel(data_path+current_stops_data, sheet_name = "Sheet2")
fut_stop_df = pd.read_excel(data_path+future_stops_data, sheet_name = "Sheet2")

In [123]:
stop_df.head()

,Stops,Coordinates,Line,Community
0,69th street,"51.037317,-114.187902",Blue Line,Christie Park\nSpringbank Hill\nSignal Hill\nA...
1,Sirocco,"51.038353, -114.169128",Blue Line,Christie Park\nSignal Hill
2,45th Street,"51.03776,-114.153425",Blue Line,Westgate\nRosscarrock\nGlendale
3,Westbrook,"51.041596,-114.124162",Blue Line,Shaganappi\nRosscarrock\nKillarney
4,Shagannappi Point,"51.041724,-114.124038",Blue Line,Shaganappi


In [124]:
stop_df['Community'] = stop_df['Community'].str.rstrip('\n').str.replace('\n',',')

In [125]:
stop_df.head()

,Stops,Coordinates,Line,Community
0,69th street,"51.037317,-114.187902",Blue Line,"Christie Park,Springbank Hill,Signal Hill,Aspe..."
1,Sirocco,"51.038353, -114.169128",Blue Line,"Christie Park,Signal Hill"
2,45th Street,"51.03776,-114.153425",Blue Line,"Westgate,Rosscarrock,Glendale"
3,Westbrook,"51.041596,-114.124162",Blue Line,"Shaganappi,Rosscarrock,Killarney"
4,Shagannappi Point,"51.041724,-114.124038",Blue Line,Shaganappi


In [126]:
# service_dict = {}
comm_list = []
red_list = []
blue_list = []
orange_list = []
yellow_list = []
purple_list = []
teal_list = []

In [127]:
for index, row in stop_df.iterrows():
    if 'Line' in row['Line']:
        if 'Red' in row['Line']:
            red_list.extend([com for com in row['Community'].split(',')])
        else:
            blue_list.extend([com for com in row['Community'].split(',')])
    else:
        if 'Orange' in row['Line']:
            orange_list.extend([com for com in row['Community'].split(',')])
        elif 'Purple' in row['Line']:
            purple_list.extend([com for com in row['Community'].split(',')])
        elif 'Teal' in row['Line']:
            teal_list.extend([com for com in row['Community'].split(',')])
        else:
            yellow_list.extend([com for com in row['Community'].split(',')])

In [128]:
comm_list = list(set(red_list+blue_list+orange_list+purple_list+teal_list+yellow_list))
clean_com_list = [item for item in comm_list if item.strip()]
service_df = pd.DataFrame(clean_com_list, columns=["Community"]).sort_values(by="Community")

In [129]:
service_df

,Community
21,Acadia
38,Altadore
54,Applewood Park
65,Arbour Lake
26,Aspen Woods
...,...
93,Whitehorn
74,Willow Park
0,Winston Heights/Mountview
1,Woodbine


In [130]:
service_df["Red Trips"] = service_df["Community"].isin(red_list) * red_trips
service_df["Red Freq"] = service_df["Community"].isin(red_list) * red_freq
service_df["Blue Trips"] = service_df["Community"].isin(blue_list) * blue_trips
service_df["Blue Freq"] = service_df["Community"].isin(blue_list) * blue_freq
service_df["Orange Trips"] = service_df["Community"].isin(orange_list) * orange_trips
service_df["Orange Freq"] = service_df["Community"].isin(orange_list) * orange_freq
service_df["Yellow Trips"] = service_df["Community"].isin(yellow_list) * yellow_trips
service_df["Yellow Freq"] = service_df["Community"].isin(yellow_list) * yellow_freq
service_df["Teal Trips"] = service_df["Community"].isin(teal_list) * teal_trips
service_df["Teal Freq"] = service_df["Community"].isin(teal_list) * teal_freq
service_df["Purple Trips"] = service_df["Community"].isin(purple_list) * purple_trips
service_df["Purple Freq"] = service_df["Community"].isin(purple_list) * purple_freq

In [131]:
service_df

,Community,Red Trips,Red Freq,Blue Trips,Blue Freq,Orange Trips,Orange Freq,Yellow Trips,Yellow Freq,Teal Trips,Teal Freq,Purple Trips,Purple Freq
21,Acadia,226,10.0,0,0.0,0,0.0,0,0.0,61,12.0,0,0.0
38,Altadore,0,0.0,0,0.0,0,0.0,75,9.5,0,0.0,0,0.0
54,Applewood Park,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,66,25.0
65,Arbour Lake,226,10.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
26,Aspen Woods,0,0.0,193,4.0,0,0.0,0,0.0,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,Whitehorn,0,0.0,193,4.0,98,7.0,0,0.0,0,0.0,0,0.0
74,Willow Park,226,10.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
0,Winston Heights/Mountview,0,0.0,0,0.0,98,7.0,0,0.0,0,0.0,0,0.0
1,Woodbine,0,0.0,0,0.0,0,0.0,75,9.5,0,0.0,0,0.0


In [132]:
# Merge the two DataFrames on the 'Community' and 'Community Name' columns
merged_data = pd.merge(service_df, comm_data, left_on='Community', right_on='Community Name', how='right')

# Update the 'Population' column in sup_fc with the corresponding values from comm_data
service_df['Population'] = service_df['Community'].apply(
    lambda x: merged_data.loc[merged_data['Community Name'] == x, 'Population in private households'].iloc[0]
    if x in merged_data['Community Name'].tolist() else 0
)

# dropping rows that have no population:
service_df = service_df[service_df['Population'] != 0]

In [133]:
service_df["Frequency Index"] = service_df.filter(like="Freq").sum(axis=1)

In [134]:
service_df['Total Trips'] = service_df.filter(regex="Red Trips|Blue Trips").sum(axis=1)*train_capacity + service_df.filter(regex="Orange Trips|Yellow Trips|Teal Trips|Yellow Trips|Purple Trips").sum(axis=1)*bus_capacity
service_df["Capacity Index"] = service_df['Total Trips']/service_df.filter(like='Population').sum(axis=1)

In [135]:
min_freq = min(service_df['Frequency Index'])
max_freq = max(service_df['Frequency Index'])
min_cap = min(service_df['Capacity Index'])
max_cap = max(service_df['Capacity Index'])

In [136]:
service_df['SS Frequency Index'] = (service_df['Frequency Index']-min_freq)/(max_freq-min_freq)
service_df['SS Capacity Index'] = (service_df['Capacity Index']-min_cap)/(max_cap-min_cap)

In [137]:
service_df

,Community,Red Trips,Red Freq,Blue Trips,Blue Freq,Orange Trips,Orange Freq,Yellow Trips,Yellow Freq,Teal Trips,Teal Freq,Purple Trips,Purple Freq,Population,Frequency Index,Total Trips,Capacity Index,SS Frequency Index,SS Capacity Index
21,Acadia,226,10.0,0,0.0,0,0.0,0,0.0,61,12.0,0,0.0,9915,22.0,139565,14.076147,0.404494,0.062027
38,Altadore,0,0.0,0,0.0,0,0.0,75,9.5,0,0.0,0,0.0,7290,9.5,4875,0.668724,0.123596,0.001630
54,Applewood Park,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,66,25.0,6830,25.0,4290,0.628111,0.471910,0.001447
65,Arbour Lake,226,10.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,10335,10.0,135600,13.120464,0.134831,0.057722
26,Aspen Woods,0,0.0,193,4.0,0,0.0,0,0.0,0,0.0,0,0.0,9435,4.0,115800,12.273450,0.000000,0.053906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,Whitehorn,0,0.0,193,4.0,98,7.0,0,0.0,0,0.0,0,0.0,11085,11.0,122170,11.021200,0.157303,0.048265
74,Willow Park,226,10.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,5090,10.0,135600,26.640472,0.134831,0.118626
0,Winston Heights/Mountview,0,0.0,0,0.0,98,7.0,0,0.0,0,0.0,0,0.0,3605,7.0,6370,1.766990,0.067416,0.006577
1,Woodbine,0,0.0,0,0.0,0,0.0,75,9.5,0,0.0,0,0.0,8745,9.5,4875,0.557461,0.123596,0.001129


In [138]:
service_df.loc[service_df['SS Frequency Index']==0].shape

(10, 19)

In [139]:
service_df.loc[service_df['Population'] == 0].shape

(0, 19)